In [9]:
import numpy as np

from strat import MACDandEMA

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="15m",
    candles_to_dl=3000,
)

In [11]:
macd_strat = MACDandEMA(
    long_short="long",
    ema_length=np.arange(200, 401, 200),
    fast_length=np.arange(10, 21, 10),
    macd_below=np.array([0]),
    signal_smoothing=np.arange(5, 16, 10),
    slow_length=np.arange(30, 61, 30),
)

In [12]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)



In [13]:
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [14]:
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([5]),
    sl_based_on_add_pct=np.array([0.1, 0.25]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [15]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 16
Total order settings to test: 16
Total combinations of settings to test: 256
Total candles: 3,000
Total candles to test: 768,000


In [16]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
159,9,15,13.0,8,5,106.331,61.538,0.894,126.920,2063.314,1063.314
223,13,15,11.0,7,4,167.964,63.636,0.887,150.004,2679.639,1679.639
151,9,7,15.0,8,7,114.670,53.333,0.883,176.867,2146.703,1146.703
157,9,13,13.0,8,5,75.796,61.538,0.882,115.514,1757.965,757.965
221,13,13,11.0,7,4,136.589,63.636,0.875,135.004,2365.887,1365.887
222,13,14,12.0,7,5,130.694,58.333,0.839,130.789,2306.940,1306.940
149,9,5,15.0,8,7,63.539,53.333,0.836,149.299,1635.394,635.394
158,9,14,14.0,8,6,71.427,57.143,0.827,109.120,1714.270,714.270
215,13,7,13.0,7,6,162.378,53.846,0.826,188.639,2623.782,1623.782
220,13,12,12.0,8,4,97.721,66.667,0.815,120.084,1977.209,977.209


In [17]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
    dos_index=15,
    ind_set_index=13,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 0,
    risk_account_pct_size = 0.03,
    risk_reward = 5.0,
    sl_based_on_add_pct = 0.0025,
    sl_based_on_lookback = 50,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.01,
    trail_sl_when_pct = 0.02,
)


In [20]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,13,15,421,1705971600000,2024-01-23 01:00:00,EntryFilled,1000.000,959.818,1937.111,40.182,...,1977.293,39869.4,NaN,0.050,1977.293,NaN,1.4,39312.0,7.7,42930.7
1,13,15,453,1706000400000,2024-01-23 09:00:00,StopLossFilled,969.755,969.755,NaN,NaN,...,NaN,NaN,39312.0,NaN,NaN,-30.245,NaN,NaN,NaN,NaN
2,13,15,604,1706136300000,2024-01-24 22:45:00,EntryFilled,969.755,928.508,2376.814,41.247,...,2418.061,39825.7,NaN,0.061,2418.061,NaN,1.1,39395.6,6.1,42250.4
3,13,15,682,1706206500000,2024-01-25 18:15:00,EntryFilled,969.755,848.630,7031.149,121.125,...,2357.399,39932.7,NaN,0.120,4775.460,NaN,1.1,39441.7,6.2,42318.3
4,13,15,713,1706234400000,2024-01-26 02:00:00,EntryFilled,969.755,727.187,13886.395,242.568,...,2322.354,39928.2,NaN,0.178,7097.814,NaN,1.1,39441.7,6.2,42358.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,13,15,2764,1708080300000,2024-02-16 10:45:00,EntryFilled,3039.754,2680.243,18329.310,359.511,...,4965.811,52102.1,NaN,0.229,11890.264,NaN,1.3,51209.3,7.7,55929.6
111,13,15,2808,1708119900000,2024-02-16 21:45:00,EntryFilled,3039.754,2288.055,43604.915,751.699,...,14137.040,52021.2,NaN,0.501,26027.304,NaN,0.9,51486.8,5.3,54720.4
112,13,15,2835,1708144200000,2024-02-17 04:30:00,EntryFilled,3039.754,1732.443,81500.680,1307.311,...,13175.772,52007.1,NaN,0.754,39203.076,NaN,0.8,51547.7,4.7,54423.1
113,13,15,2858,1708164900000,2024-02-17 10:15:00,StopLossFilled,2679.639,2679.639,NaN,NaN,...,NaN,NaN,51547.7,NaN,NaN,-360.115,NaN,NaN,NaN,NaN
